In [ ]:
import os, sys
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
from z3 import set_param

os.chdir("/home/yizhou7/mariposa")

from utils.system_utils import log_info
from debugger.trace_analyzer import shorten_qid
from debugger3 import Debugger3

UNSTABLE_10_SECS = {
    "data/projs/flowcert/base.z3/permission.14.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v__LikesBetree.3.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v.18.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_eventually_always_no_delete_resource_request_msg_in_flight.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_from_after_get_stateful_set_step_to_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.6.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_every_resource_create_request_implies_at_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__daemon_set_match__lemma_from_after_get_daemon_set_step_to_after_update_daemon_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_object_in_response_at_after_create_resource_step_is_same_as_etcd.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_stateful_set_state_matches_at_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__validation__lemma_always_stateful_set_in_create_request_msg_satisfies_unchangeable.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.5.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2",
    "data/projs/verismo.dedup/base.z3/security__monitor.21.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.5.smt2",
    "data/projs/verismo.dedup/base.z3/arch__ptram__ptram_p2.smt2",
    "data/projs/verismo.dedup/base.z3/bsp__ap.3.smt2",
    "data/projs/verismo.dedup/base.z3/snp__cpuid.17.smt2",
    "data/projs/verismo.dedup/base.z3/snp__ghcb__proto_page.141.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.20.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.4.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init_alloc.1.smt2",
    "data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.4.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__span_queue_delete.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.12.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.16.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__logimpl_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__start_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg.69.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.7.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_span_free_coalesce_before.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.15.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.9.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.10.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__layout_v.30.smt2",
    "data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2",
}

# def load_scores():
#     scores = dict()

#     for line in open("/home/yizhou7/mariposa/doc/scores.csv", "r"):
#         parts = line.strip().split(",")
#         query = parts[0]
#         scores[query] = [int(s) for s in parts[1:]]
        
#     return scores

# scores = load_scores()

# print("total unstable:", len(UNSTABLE_10_SECS))
# print("no proofs:", len(UNSTABLE_10_SECS - set(scores.keys())))

# no_fixes = set()
# bad_scores = set()

# for query, score in scores.items():
#     if score[0] == 0:
#         no_fixes.add(query)
#     elif all([s == 0 for s in score[1:]]):
#         bad_scores.add(query)

# print("no fixes:", len(no_fixes))
# print("bad scores:", len(bad_scores))

# for query in sorted(bad_scores):
#     print(query, scores[query][0])

In [ ]:
from debugger.edit_info import EditAction

dbg = Debugger3("data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2")

In [ ]:
from debugger.edit_info import EditInfo
import random
from base.solver import RCode

def debugger_erase_search(dbg: Debugger3):
    actions = dbg.differ.actions
    erasable = []

    for qid, action in actions.items():
        if action == EditAction.ERASE:
            erasable.append(qid)

    print("total erasable:", len(erasable))
    ei: EditInfo = dbg.test_edit(set(erasable))
    print("ideal:", ei.rcode, ei.time)

    current = random.sample(erasable, 1)
    ei: EditInfo = dbg.test_edit(set(current))
    print("current:", ei.rcode, ei.time)

    if ei.rcode != RCode.UNSAT:
        print("bad: ", current)

    # print("current:", current)
    # print(ei.rcode, ei.time)

debugger_erase_search(dbg)


In [2]:
import os, sys
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
from z3 import set_param
import networkx as nx

os.chdir("/home/yizhou7/mariposa")

from debugger.proof_analyzer import ProofAnalyzer


proof_file = "dbg/815f69b161/proofs/shuffle.13565831226465156427.proof"
pa = ProofAnalyzer(proof_file)
# pa.sanity_check_proof_nodes()
proof_graph = pa.build_proof_graph()
# for nid in proof_graph.nodes:
#     # print(nid, end=" ")
#     pa.pprint_node(nid)

In [5]:
print(len(proof_graph.nodes), len(proof_graph.edges))
print(len(pa.nodes), len(pa.edges))

scores = nx.betweenness_centrality(proof_graph)

1986 38970
16829 40817


In [7]:
scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

for nid, score in scores[:10]:
    pa.pprint_node(nid, 4)
    print(score)

(lemma h!3b15536f h!bb61cd0c)
0.010748909031345245
(lemma h!d662eeab h!3be770b9)
0.0061903822195114476
(lemma h!828c022b h!5cc5fff7)
0.004108873777550561
(lemma h!943ceb04 h!46c7c2f1)
0.0034018502068316227
(th-lemma h!b3fa7ce2 h!026b2fea h!294e385b h!848e6f52 h!a7a96eab h!6e5bf155 h!24807f32 h!7b583c8c h!96bb9ec3 h!e02d73a8 h!6038fa50 h!b874b94a h!7cb6efb9)
0.0029393578922048496
(lemma h!38c9fee8 h!f8d2ec80)
0.0028776552026785634
(th-lemma h!0a71a6a5 h!6e5bf155 h!1f692422 h!f2fe2c07 h!b7507397 h!8ca8213f h!49ab4921 h!471395de h!1da15a25 h!7cb6efb9)
0.0025654619225605247
(lemma h!33ed2d4e h!d3e2e5c8)
0.0025144238954214987
(th-lemma h!bc8584b3 h!7790a98e h!396af633 h!8a257cbe h!dfa00617 h!cc2b3167 h!aa7c9574 h!0ca5944e h!bbe6f935 h!4f4592ea h!49dd6485 h!e0ffb562 h!db79cfd4 h!455309f2 h!f61cd6a8 h!5a534406 h!72d321d9 h!5ab99a43 h!e5e0ff37 h!049aee97 h!00e71a9b h!08a46c23 h!7cb6efb9)
0.0022854752376696183
(lemma h!a2e6d1c0 h!7d67d183)
0.002078273966712712
